In [1]:
'''
Build a Dashboard Application with Plotly Dash
In this lab, you will be building a Plotly Dash application for users to perform interactive visual analytics on SpaceX launch data in
real-time.
This dashboard application contains input components such as a dropdown list and a range slider to
interact with a pie chart and a scatter point chart. You will be guided to build this dashboard application via the following tasks:
    TASK 1: Add a Launch Site Drop-down Input Component
    TASK 2: Add a callback function to render success-pie-chart based on selected site dropdown
    TASK 3: Add a Range Slider to Select Payload
    TASK 4: Add a callback function to render the success-payload-scatter-chart scatter plot
Note:Please take screenshots of the Dashboard and save them. Further upload your notebook to github.
The github url and the screenshots are later required in the presentation slides.
'''

'\nBuild a Dashboard Application with Plotly Dash\nIn this lab, you will be building a Plotly Dash application for users to perform interactive visual analytics on SpaceX launch data in\nreal-time.\nThis dashboard application contains input components such as a dropdown list and a range slider to\ninteract with a pie chart and a scatter point chart. You will be guided to build this dashboard application via the following tasks:\n    TASK 1: Add a Launch Site Drop-down Input Component\n    TASK 2: Add a callback function to render success-pie-chart based on selected site dropdown\n    TASK 3: Add a Range Slider to Select Payload\n    TASK 4: Add a callback function to render the success-payload-scatter-chart scatter plot\nNote:Please take screenshots of the Dashboard and save them. Further upload your notebook to github.\nThe github url and the screenshots are later required in the presentation slides.\n'

In [2]:
# in terminal....
#python3.8 -m pip install pandas dash
#wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"
#. wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/spacex_dash_app.py"


In [3]:
  # TASK 1: Add a dropdown list to enable Launch Site selection

In [4]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

step = 1000
max_value = 10000
marks = {i: str(i) for i in range(0, max_value + step, step)}

# Create a dash application
app = dash.Dash(__name__)

site_list= spacex_df['Launch Site'].unique()
      
# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                              
                                dcc.Dropdown(id='site-dropdown',
                                            options=[{'label': 'All Sites', 'value': 'ALL'}] + [{'label': i, 'value': i} for i in site_list],
                                            #value='ALL',
                                            placeholder="All Sites",
                                            searchable=True
                                            ),
   
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(id='payload-slider',
                                                    min=0, max=10000, step=1000,
                                                    #marks={0: '0', 100: '100'},
                                                    marks = {i: str(i) for i in range(int(min_payload), int(max_payload), 500)},
                                                    value=[min_payload, max_payload]),


                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])


C:\Users\yhuang\AppData\Local\Temp\ipykernel_19368\2225602595.py:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
C:\Users\yhuang\AppData\Local\Temp\ipykernel_19368\2225602595.py:5: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


In [5]:
# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output

In [6]:
# Function decorator to specify function input and output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
            Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        #df= spacex_df.groupby('Launch Site')['class'].sum().reset_index()     
        fig = px.pie(spacex_df, values='class',
                    names='Launch Site',
                    title='Total success launches by sites')
        return fig
    else:
        # return the outcomes piechart for a selected site
        filtered_df = spacex_df.loc[spacex_df['Launch Site']==entered_site]
        fig = px.pie(filtered_df, 
                    names='class',
                    title='Success launches by site')
        return fig

TASK 4: Add a callback function to render the success-payload-scatter-chart scatter plot
Next, we want to plot a scatter plot with the x axis to be the payload and the y axis to be the launch outcome (i.e., class column).
As such, we can visually observe how payload may be correlated with mission outcomes for selected site(s).
In addition, we want to color-label the Booster version on each scatter point so that we may
observe mission outcomes with different boosters.

A If-Else statement to check if ALL sites were selected or just a specific launch site was selected
If ALL sites are selected, render a scatter plot to display all values for variable Payload Mass (kg) and variable class.
In addition, the point color needs to be set to the booster version i.e., color="Booster Version Category"
If a specific launch site is selected, you need to filter the spacex_df first, and render a scatter chart to show
values Payload Mass (kg) and class for the selected site, and color-label the point using Boosster Version Category likewise.


In [7]:
# Function decorator to specify function input and output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
           [Input(component_id='site-dropdown', component_property='value'), Input(component_id="payload-slider", component_property="value")])
def get_scatter_chart(entered_site, payload_range):
    if entered_site == 'ALL':  
        df1= spacex_df[(spacex_df['Payload Mass (kg)']>=min(payload_range)) & (spacex_df['Payload Mass (kg)']<= max(payload_range)) ]
        fig = px.scatter(df1, y='class',
                x='Payload Mass (kg)', color="Booster Version Category",
                title='Correlation between payload and success - All sites')
        return fig
    else:
        # return the outcomes piechart for a selected site
        filtered_df = spacex_df.loc[spacex_df['Launch Site']==entered_site]
        df1= spacex_df[(filtered_df['Payload Mass (kg)']>=min(payload_range)) & (spacex_df['Payload Mass (kg)']<= max(payload_range)) ]
        fig = px.scatter(df1, y='class',
                x='Payload Mass (kg)', color="Booster Version Category",
                title='Correlation between payload and success by site')
        return fig

In [8]:

# Run the app
if __name__ == '__main__':
    app.run_server()

